In [1]:
import sys
# !pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Load dataset
file_path = "chennai-ground-water-levels-2023.csv"  # Replace with actual file path
df = pd.read_csv(file_path)

In [10]:
# Columns for groundwater levels
months = ["Jan ", "Feb", "Mar", "Apr ", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
df_filtered = df[["Location"] + months]

In [11]:
import sys
print(sys.executable)

C:\Users\rumma\AppData\Local\Programs\Python\Python313\python.exe


In [12]:
!{sys.executable} -m pip install tensorflow


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\rumma\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow


In [3]:
!python -m pip show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\rumma\AppData\Roaming\Python\Python312\site-packages
Requires: tensorflow-intel
Required-by: 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Load dataset
file_path = "chennai-ground-water-levels-2023.csv"  # Replace with actual file path
df = pd.read_csv(file_path)

# Columns for groundwater levels
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
df_filtered = df[["Location"] + months]

# Select a specific location for forecasting (e.g., Kathivakkam)
location = "Kathivakkam"
df_location = df_filtered[df_filtered["Location"] == location].iloc[:, 1:].mean().to_frame()
df_location.columns = ["Groundwater Level"]
df_location.index = pd.date_range(start="2023-01", periods=12, freq="M")  # Monthly index

# Normalize Data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_location)

# Prepare Data for LSTM
sequence_length = 3  # Using last 3 months to predict the next month
X, y = [], []
for i in range(len(data_scaled) - sequence_length):
    X.append(data_scaled[i:i + sequence_length])
    y.append(data_scaled[i + sequence_length])

X, y = np.array(X), np.array(y)

# Split into training & testing sets
train_size = int(len(X) * 0.8)
X_train, X_test, y_train, y_test = X[:train_size], X[train_size:], y[:train_size], y[train_size:]

# Build LSTM Model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)),
    LSTM(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=100, batch_size=8, verbose=1)

# Predict Next 6 Months
future_predictions = []
input_seq = data_scaled[-sequence_length:].reshape(1, sequence_length, 1)

for _ in range(6):  # Predict next 6 months
    next_value = model.predict(input_seq)[0]
    future_predictions.append(next_value)
    input_seq = np.append(input_seq[:, 1:, :], [[next_value]], axis=1)  # Slide window

# Convert back to original scale
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Plot Results
plt.figure(figsize=(10, 5))
plt.plot(df_location.index, df_location["Groundwater Level"], marker='o', label="Actual Data")
plt.plot(pd.date_range(start="2024-01", periods=6, freq="M"), future_predictions, marker='x', linestyle="dashed", label="Predicted")
plt.xlabel("Month")
plt.ylabel("Groundwater Level (m)")
plt.title(f"LSTM Groundwater Level Prediction for {location}")
plt.legend()
plt.grid()
plt.show()
